In [1]:
from glob import glob
from os.path import join as pjoin

import gym
import textworld.gym
from textworld import EnvInfos

GAMES_PATH = "/root/notebooks/Guntis/train"  # This assumes `sample_games.zip` was first unzipped.
gamefiles = glob(pjoin(GAMES_PATH, "*.ulx"))
print("Found {} games.".format(len(gamefiles)))

Found 4440 games.


In [2]:
from typing import Union


class Room(object):
    def __init__(self, name: str, directions: Union[dict, list]):
        self.name = name
        if isinstance(directions, (tuple, list)):
            self.directions = dict.fromkeys(directions)
        else:
            self.directions = directions
        self.observation = None

    def __repr__(self):
        directions = {direction: room.name if room is not None else None for (direction, room) in
                      self.directions.items()}
        return f'name={self.name}, directions={directions}'

    __str__ = __repr__

In [3]:
import random

#from room import Room


def opposite_dir(direction):
    if direction == "east":
        return "west"
    elif direction == "west":
        return "east"
    elif direction == "north":
        return "south"
    else:
        return "north"


class RoomSearch(object):
    """
    Currently a depth first search
    but it should be made smarter to know things like the Kitchen is near the Living Room.
    """

    def __init__(self, rooms: dict, current_room: Room, target_name: str):
        self._rooms = rooms
        self.current_room = current_room
        self.target_name = target_name
        self.optimal_path = None
        self._backtrack_stack = []
        self.visited = {self.current_room.name}

    def get_next_direction(self):
        # TODO Keep track if all rooms have been explored so that we can know if the target is on the map.
        if self.target_name in self._rooms and self.optimal_path is None:
            self.optimal_path = self.get_path_to(self.target_name)
        if self.optimal_path is not None and len(self.optimal_path) > 0:
            result = self.optimal_path.pop(0)
            self.prev_direction_traveled = result
            return result
        result = None
        direction_options = []
        for direction, room in self.current_room.directions.items():
            if room is not None and room.name == self.target_name:
                result = direction
                break
            elif room is None or room.name not in self.visited:
                # Don't know what that room is OR haven't visited that room.
                direction_options.append(direction)
        if result is None:
            # Nowhere specific to go.
            if len(direction_options) == 0:
                # Nowhere to go. Need to backtrack.
                if len(self._backtrack_stack) == 0:
                    # No path exists.
                    return None
                else:
                    backtrack_target = self._backtrack_stack.pop()
                    self.optimal_path = self.get_path_to(backtrack_target)
                    result = self.optimal_path.pop(0)
                    self.prev_direction_traveled = result
                    return result
            result = random.choice(direction_options)
            # If we reach a dead-end then we should come back to here.
            self._backtrack_stack.append(self.current_room.name)
        self.prev_direction_traveled = result
        return result

    def get_path_to(self, target_room_name: str):
        queue = [self.current_room]
        paths = {self.current_room.name: []}
        found = False
        while not found and len(queue) > 0:
            current_room = queue.pop(0)
            path_so_far = paths[current_room.name]
            for direction, room in current_room.directions.items():
                if room is not None:
                    if room.name not in paths:
                        paths[room.name] = path_so_far + [direction]
                        queue.append(room)
                    if room.name == target_room_name:
                        found = True
                        break
        return paths[target_room_name]


In [4]:
import logging
import random
import re
import sys
from collections import defaultdict
from enum import Enum
from operator import itemgetter
from typing import Any, Dict, List, Optional, Union

from textworld import EnvInfos

try:
    from room import Room
    from room_search import opposite_dir, RoomSearch
except:
    pass

_directions = ["north", "east", "south", "west"]

_all_ingredients = """
black pepper
bell pepper
carrot
cilantro
egg
lettuce
salt
sliced carrot
sugar
pepper
potato
red hot pepper
red onion
""".split('\n')
_all_ingredients = list(filter(None, map(str.strip, _all_ingredients)))

_max_capacity = 7
"""
The maximum number of objects that can be held.
"""

_rooms_with_ingredients = [
    "Backyard",
    "Garden",
    "Kitchen",
    "Pantry",
    "Supermarket",
]
_ingredient_to_rooms = defaultdict(lambda: _rooms_with_ingredients, {
    "egg": ["Kitchen", "Supermarket"],
})
"""
The rooms that each ingredient can be found in.
"""


class Feature(Enum):
    OBSERVING_KITCHEN = 0
    COOKBOOK_PRESENT = 1
    SEEN_COOKBOOK = 2
    COOKBOOK_SHOWING = 3

    CURRENT_ROOM = 4
    """
    String for current room name.
    """

    DONE_INIT_INVENTORY_CHECK = 5
    INVENTORY_SHOWING = 6

    NUM_ITEMS_HELD = 7
    HOLDING_KNIFE = 8

    FOUND_ALL_INGREDIENTS = 9

    OPENED_FRIDGE = 10

    CARRYING_TOO_MUCH = 11

    CANT_SEE_SUCH_THING = 12

    STARTED_COOKING = 13

    YOU_TAKE = 14

    BBQ_PRESENT = 15

    NEED_TO_OPEN_FIRST = 16
    YOU_OPENED_DOOR = 17

    def __repr__(self):
        return self.name


#######################################
# Functions For Features
#######################################
_closed_door_pattern = re.compile(r'\bclosed (?P<item>[^\s]+ door) leading (?P<direction>[^\s.,!?]+)\b', re.IGNORECASE)
_need_to_open_door_pattern = re.compile(r'You have to (?P<task>open .* door) first.')
_you_open_door_pattern = re.compile(r'You open (.* door).')


def _feat(qualifier, term):
    return (qualifier, term)


def _carrying_feat(item):
    return _feat('carrying', item)


def _direction_closed_feat(direction):
    return _feat('closed', direction)


def _direction_feat(direction):
    return _feat('available', direction)


def _ingredient_feat(ingredient):
    return _feat('ingredient', ingredient)


def _ingredient_present_feat(ingredient):
    return _feat('present', ingredient)


def _recipe_step_feat(recipe_step_index, recipe_step):
    return ('recipe_step', recipe_step_index, recipe_step)


def _get_rdfs_with_qualifier(rdfs: Dict, qualifier: Union[str, tuple]) -> List:
    result = []
    if not isinstance(qualifier, tuple):
        qualifier = (qualifier,)
    for feat, val in rdfs.items():
        if val != False and isinstance(feat, tuple) and feat[:len(qualifier)] == qualifier:
            if len(feat) == len(qualifier) + 1:
                result.append(feat[len(qualifier)])
            else:
                result.append(feat[len(qualifier):])
    return result


def _get_all_present_ingredients(rdfs: Dict) -> List[str]:
    return _get_rdfs_with_qualifier(rdfs, 'present')


def _get_all_required_ingredients(rdfs: Dict) -> List[str]:
    return _get_rdfs_with_qualifier(rdfs, 'ingredient')

def _get_all_recipe_ingredients(rdfs: Dict) -> List[str]:
    return _get_rdfs_with_qualifier(rdfs, 'recipe-ingredient')

def _get_carrying(rdfs: Dict):
    return _get_rdfs_with_qualifier(rdfs, 'carrying')


def _get_recipe_steps(rdfs: Dict):
    """
    :param rdfs: The features.
    :return: The recipe steps in order.
    """
    recipe_rdfs = _get_rdfs_with_qualifier(rdfs, 'recipe_step')
    recipe_rdfs.sort(key=itemgetter(0))
    return [step[1] for step in recipe_rdfs]


def _remove_recipe_step(rdfs: Dict, recipe_step: str):
    recipe_rdfs = _get_rdfs_with_qualifier(rdfs, 'recipe_step')
    for recipe_feat in recipe_rdfs:
        if recipe_feat[1] == recipe_step:
            rdfs[_recipe_step_feat(recipe_feat[0], recipe_step)] = False


#######################################
# END Functions For Features
#######################################

def _gather_inventory(ob):
    result = []
    for line in ob.split('\n'):
        line = line.strip()
        if line.startswith("You are carrying:") or line.startswith("You are carrying nothing."):
            continue
        if line:
            prefixes = ("a ", "an ", "some ", "raw ")
            modified = True
            while modified:
                modified = False
                for prefix in prefixes:
                    if line.startswith(prefix):
                        modified = True
                        line = line[len(prefix):]
            result.append(line)

    return result


# Cooking Patterns
_fry_pattern = re.compile('^fry the (?P<ingredient>.*)', re.IGNORECASE)
_grill_pattern = re.compile('^grill the (?P<ingredient>.*)', re.IGNORECASE)
_roast_pattern = re.compile('^roast the (?P<ingredient>.*)', re.IGNORECASE)

_fried_pattern = re.compile('^fried (?P<ingredient>.*)', re.IGNORECASE)
_grilled_pattern = re.compile('^grilled (?P<ingredient>.*)', re.IGNORECASE)
_roasted_pattern = re.compile('^roasted (?P<ingredient>.*)', re.IGNORECASE)

# Cutting Patterns
_chop_pattern = re.compile('^chop the (?P<ingredient>.*)', re.IGNORECASE)
_dice_pattern = re.compile('^dice the (?P<ingredient>.*)', re.IGNORECASE)
_slice_pattern = re.compile('^slice the (?P<ingredient>.*)', re.IGNORECASE)

_chopped_pattern = re.compile('^chopped (?P<ingredient>.*)', re.IGNORECASE)
_diced_pattern = re.compile('^diced (?P<ingredient>.*)', re.IGNORECASE)
_sliced_pattern = re.compile('^sliced (?P<ingredient>.*)', re.IGNORECASE)

_require_knife_pattern = re.compile('^(chop|dice|slice) ', re.IGNORECASE)


def _commandify_recipe_step(recipe_step):
    result = recipe_step
    m = _fry_pattern.match(recipe_step)
    if m:
        result = f"cook {m['ingredient']} with stove"
    elif _grill_pattern.match(recipe_step):
        m = _grill_pattern.match(recipe_step)
        result = f"cook {m['ingredient']} with BBQ"
    elif _roast_pattern.match(recipe_step):
        m = _roast_pattern.match(recipe_step)
        result = f"cook {m['ingredient']} with oven"

    return result


def _get_ingredients_present(observation, ingredient_candidates):
    result = []
    covered = set()
    for ingredient in sorted(ingredient_candidates, key=len, reverse=True):
        m = re.search(r'\b{}\b'.format(ingredient), observation, re.IGNORECASE)
        if m and m.start() not in covered:
            covered.update(range(m.start(), m.end()))
            result.append(ingredient)
    return result


def _get_recipe_steps_to_make(ingredient) -> set:
    result = set()
    if _fried_pattern.match(ingredient):
        m = _fried_pattern.match(ingredient)
        result.add(f"fry the {_base_ingredient(m['ingredient'])}")
        result.update(_get_recipe_steps_to_make(m['ingredient']))
    if _grilled_pattern.match(ingredient):
        m = _grilled_pattern.match(ingredient)
        result.add(f"grill the {_base_ingredient(m['ingredient'])}")
        result.update(_get_recipe_steps_to_make(m['ingredient']))
    if _roasted_pattern.match(ingredient):
        m = _roasted_pattern.match(ingredient)
        result.add(f"roast the {_base_ingredient(m['ingredient'])}")
        result.update(_get_recipe_steps_to_make(m['ingredient']))
    if _chopped_pattern.match(ingredient):
        m = _chopped_pattern.match(ingredient)
        result.add(f"chop the {_base_ingredient(m['ingredient'])}")
        result.update(_get_recipe_steps_to_make(m['ingredient']))
    if _diced_pattern.match(ingredient):
        m = _diced_pattern.match(ingredient)
        result.add(f"dice the {_base_ingredient(m['ingredient'])}")
        result.update(_get_recipe_steps_to_make(m['ingredient']))
    if _sliced_pattern.match(ingredient):
        m = _sliced_pattern.match(ingredient)
        result.add(f"slice the {_base_ingredient(m['ingredient'])}")
        result.update(_get_recipe_steps_to_make(m['ingredient']))
    return result


def _recipe_step_to_ingredient(recipe_step):
    result = None
    if _chop_pattern.match(recipe_step):
        m = _chop_pattern.match(recipe_step)
        result = f"chopped {m['ingredient']}"
    elif _dice_pattern.match(recipe_step):
        m = _dice_pattern.match(recipe_step)
        result = f"diced {m['ingredient']}"
    elif _slice_pattern.match(recipe_step):
        m = _slice_pattern.match(recipe_step)
        result = f"sliced {m['ingredient']}"
    elif _fry_pattern.match(recipe_step):
        m = _fry_pattern.match(recipe_step)
        result = f"fried {m['ingredient']}"
    elif _grill_pattern.match(recipe_step):
        m = _grill_pattern.match(recipe_step)
        result = f"grilled {m['ingredient']}"
    elif _roast_pattern.match(recipe_step):
        m = _roast_pattern.match(recipe_step)
        result = f"roasted {m['ingredient']}"
    return result


def _requires_knife(recipe_step):
    return _require_knife_pattern.match(recipe_step) is not None


def _base_ingredient(ingredient: str) -> str:
    result = ingredient
    prefixes = ("chopped ", "diced ", "sliced ",
                "fried ", "grilled ", "roasted ")
    prefixes += ("chop ", "dice ", "slice ",
                "fry ", "grill ", "roast ", "raw ")

    updated = True
    while updated:
        updated = False
        for prefix in prefixes:
            if result.startswith(prefix):
                result = result[len(prefix):]
                updated = True

    return result


class CustomAgent:
    """ Template agent for the TextWorld competition. """

    def __init__(self) -> None:
        self._initialized = False
        self._epsiode_has_started = False
        self.reb = 1
        self.current_room = None
        self.take_ingredients = 0 # 1- in progress, 2- done => eat meal

    def train(self) -> None:
        """ Tell the agent it is in training mode. """


        determiner_re = re.compile(r'(^|\s)((a|an|the|few|some|raw)\s+)+', re.I)

        def parse_inventory(inventory_string):
            inventory = []
            for line in inventory_string.splitlines():
                if not line:
                    continue
                if line.startswith('  '):
                    inventory.append(determiner_re.sub(r'\1', line.strip()))
            return inventory

        def parse_paragraphs(text):
            return list(filter(None, (list(filter(None, g)) for k,g in groupby(text.splitlines() if type(text) is str else text, key=lambda item: bool(item.strip())))))

        def normalize_ingredients(ingredients):
            if type(ingredients) in (list, tuple):
                return [normalize_ingredients(ingredient) for ingredient in ingredients]
            if type(ingredients) is str:
                return determiner_re.sub(r'\1', ingredients.strip())
            return ingredients

        def normalize_directions(directions):
            return normalize_ingredients(directions)

        def parse_recipe(text):
            paragraphs = parse_paragraphs(text)
            assert len(paragraphs) == 3
            recipe, ingredients, directions = paragraphs
            assert recipe[0].startswith('Recipe ')
            assert ingredients[0].startswith('Ingredients:')
            assert directions[0].startswith('Directions:')
            return Dict(text=recipe[2:], ingredients=normalize_ingredients(ingredients[1:]), directions=normalize_directions(directions[1:]))
            # return Dict(text=recipe[2:], ingredients=list(map(str.strip, ingredients[1:])), directions=list(map(str.strip, directions[1:])))

        sentence_split_re = re.compile(r'(?<=[.?!])(\s+)')

        def split_sentences(text):
            if type(text) is str:
                # handle end of game message
                if text.strip().startswith('***'):
                    return [text.strip()]
                return list(filter(None, map(str.strip, sentence_split_re.split(text))))
            elif type(text) in (list, tuple):
                sentences = []
                for item in text:
                    sentences.extend(split_sentences(item))
                return sentences
            return []

        token_split_re = re.compile(r'([\s.!?,;:"\'-]+)')
        placeholder_re = re.compile(r'{([^:}]+):?([^}]+)?}')
        placeholder_fix_re = re.compile(r'\\{([^:}\\]+)(?:\\:)?([^}]+)?\\}')

        def compile_template(template):
            if type(template) is str:
                # print(template)
                # print(placeholder_fix_re.sub(r'{\1:\2}', re.escape(template)))
                counters = defaultdict(int)
                def repl(m):
                    nonlocal counters
                    kind = m.group(1)
                    counters[kind] += 1
                    # print('(?P<%s_%i>.+?)' % (m.group(1), counter))
                    return '(?P<%s_%i>.+?)' % (kind, counters[kind])
                # print('TE:', placeholder_re.sub(repl, placeholder_fix_re.sub(r'{\1:\2}', re.escape(template))))
                return Dict(compiled=re.compile('^%s$' % placeholder_re.sub(repl, placeholder_fix_re.sub(r'{\1:\2}', re.escape(template)))), string=template)
                # return re.compile(placeholder_re.sub(r'(?P<\1>.+)', placeholder_fix_re.sub(r'{\1:\2}', re.escape(template))))
                # return re.compile(placeholder_re.sub(r'(?P<\1>.+)', re.escape(template).replace(r'\{', '{').replace(r'\}', '}')))
            elif type(template) in (list, tuple):
                return [compile_template(t) for t in template]
            return template # may be an already compiled template
            # raise ValueError(f'invalid template of type {type(template)}: {template}')

        def match_templates(text, templates):
            if not templates:
                raise ValueError('invalid templates')
            # prepare templates
            if type(templates) in (list, tuple):
                if any(filter(lambda t: t is str, map(type, templates))):
                    # at least some templates are not compiled
                    templates = compile_template(templates)
            elif type(templates) is str:
                templates = [compile_template(templates)]
            else:
                templates = [templates]
            # apply templates
            if type(text) is str:
                return list(map(lambda m: m.groupdict(), filter(None, (template.compiled.match(text) for template in templates))))
            elif type(text) in (list, tuple):
                return [match_templates(item, templates) for item in text]


        def drop_placeholder_values(text):
            if type(text) in (list, tuple):
                return [drop_placeholder_values(item) for item in text]
            elif type(text) is str:
                text = placeholder_re.sub(r'{\1}', text)
            return text

        def drop_placeholder_types(text):
            if type(text) in (list, tuple):
                return [drop_placeholder_types(item) for item in text]
            elif type(text) is str:
                text = placeholder_re.sub(r'{\2}', text)
            return text

        def restore_placeholder_values(text):
            if type(text) in (list, tuple):
                return [restore_placeholder_values(item) for item in text]
            elif type(text) is str:
                text = placeholder_re.sub(r'\2', text)
            return text

        def replace_with_placeholders(text, name=None, lst=[]):
            if type(text) in (list, tuple):
                return [replace_with_placeholders(item, name, lst) for item in text]
            if type(text) is str:
                prefix = '%s:' % name if name else ''
                tokens = token_split_re.split(text)
                for item in lst:
                    item = token_split_re.split(item)
                    # TODO: varbūt vajag match pēc lemmām?
                    replace = True
                    while replace:
                        replace = False
                        if len(item) == 1:
                            if item[0] in tokens:
                                tokens[tokens.index(item[0])] = '{%s%s}' % (prefix, item[0])
                                replace = True
                        elif len(item) > 1:
                            for i in range(len(tokens)-len(item)):
                                match = True
                                for j,token in enumerate(item):
                                    if tokens[i+j] != token:
                                        match = False
                                        break
                                if match:
                                    tokens[i:i+len(item)] = ['{%s%s}' % (prefix, ''.join(item))]
                                    replace = True
                                    break
                text = ''.join(tokens)
            return text

        def parse_description(text, entities=[], verbs=[], ingredients=[]):
            text = text.splitlines()
            room = None
            for line in text:
                # handle room name
                if line.startswith('-='):
                    room = line.strip('-=').strip()
            text = filter(lambda line: not line.startswith('-='), text)
            paragraphs = [split_sentences(paragraph) for paragraph in parse_paragraphs(text)]
            # remove textworld logo
            if paragraphs and paragraphs[0][0].startswith('________'):
                paragraphs.pop(0)
            # remove QUIT line
            if paragraphs and paragraphs[-1][-1] == '>':
                paragraphs.pop()
            # TODO: entities un verbs vajag locījumus
            entities = set(entities) | set(filter(None, map(lambda e: inflect.conjugate(e, inflect.PAST, aspect=inflect.PROGRESSIVE), entities)))
            verbs = set(verbs) | set(filter(None, map(lambda e: inflect.conjugate(e, inflect.PAST, aspect=inflect.PROGRESSIVE), verbs)))
            paragraphs = replace_with_placeholders(paragraphs, 'entity', entities)
            paragraphs = replace_with_placeholders(paragraphs, 'verb', verbs)
            paragraphs = replace_with_placeholders(paragraphs, 'ingredient', ingredients)
            return Dict(room=room, paragraphs=paragraphs)

        def flatten(lst):
            if type(lst) not in (list, tuple):
                return (lst,)
            return [ee for e in lst for ee in flatten(e)]

        def nested_difference(lst, other_set):
            return list(filter(None,
                    (item if type(item) not in (list, tuple) else nested_difference(item, other_set)
                        for item in lst if type(item) in (list, tuple) or item not in other_set)))

        def nested_intersection(lst, other_set):
            return list(filter(None,
                    (item if type(item) not in (list, tuple) else nested_intersection(item, other_set)
                        for item in lst if type(item) in (list, tuple) or item in other_set)))

        # print(flatten([1,2,[4,5],8]))
        # print(nested_difference([1,2,[4,5,9,[88,77,66],[33]],8], set((2,5,77,33))))
        # print(nested_intersection([1,2,[4,5,9,[88,77,66],[33]],8], set((2,5,77,33))))
        # quit()

        def diff(left, right, enter_leave=True):
            primitive_types = (str, int, float, bool, type(None))
            if type(left) in primitive_types and type(right) in primitive_types:
                if left == right:
                    if enter_leave:
                        return Dict(leave=None, enter=None, unchanged=left)
                    return Dict(left_only=None, right_only=None, both=left)
                else:
                    if enter_leave:
                        return Dict(leave=left, enter=right, unchanged=None)
                    return Dict(left_only=left, right_only=right, both=None)
            if isinstance(left, dict) and isinstance(right, dict):
                return Dict({key: diff(left[key], right[key]) for key in left})
            left_set = set(flatten(left))
            right_set = set(flatten(right))
            # left_only = [e for e in left if e not in right_set]
            # right_only = [e for e in right if e not in left_set]
            # left_only = left_set - right_set
            # right_only = right_set - left_set
            # both = left_set & right_set
            left_only = nested_difference(left, right_set)
            right_only = nested_difference(right, left_set)
            both = nested_intersection(left, right_set) # for now we keep the structure of left list as it will be used as old value
            if enter_leave:
                # left is prev
                # right is new
                return Dict(leave=left_only, enter=right_only, unchanged=both)
            return Dict(left_only=left_only, right_only=right_only, both=both)

        def trim_unchanged(df, remove_unchanged=False):
            if type(df) in (tuple, list):
                return list(filter(None, (trim_unchanged(item, remove_unchanged) for item in df)))
            elif isinstance(df, dict):
                if 'enter' in df and 'leave' in df and 'unchanged' in df:
                    if not df.get('enter') and not df.get('leave'):
                        return None
                    elif remove_unchanged:
                        # return Dict(enter=df['enter'], leave=df['leave'])
                        r = Dict()
                        if df.get('enter'):
                            r.enter = df['enter']
                        if df.get('leave'):
                            r.leave = df['leave']
                        return r
                    else:
                        return df
                else:
                    return Dict(filter(lambda item: item[1], ((key, trim_unchanged(value, remove_unchanged)) for key,value in df.items())))
            return df

        # def print_diff(df):
        #     if type(df) in (list, tuple):
        #         for item in df:
        #             print_diff(item)
        #     elif isinstance(df, dict):
        #         if 'enter' in df and 'leave' in df and 'unchanged' in df:
        #             print()



        def parse_info(info):
            description = info.description
            inventory = parse_inventory(info.inventory)
            entities = info.entities
            verbs = info.verbs
            command_templates = info.command_templates
            admissible_commands = info.admissible_commands
            recipe = info['extra.recipe']
            recipe = parse_recipe(recipe)
            walkthrough = info['extra.walkthrough']
            description = parse_description(description, entities=entities, verbs=verbs, ingredients=recipe.ingredients)
            return Dict(
                    inventory=inventory,
                    entities=entities,
                    verbs=verbs,
                    command_templates=command_templates,
                    admissible_commands=admissible_commands,
                    walkthrough=walkthrough,
                    recipe=recipe,
                    description=description,
                )


        gamefiles = glob(os.path.join('train-data', '*'))
        data = load(gamefiles[0])

        ppw = 120

        for gver in data:
            kind = gver[0]
            history = gver[1:]
            print('---------- %s ----------' % 'NEW RUN')
            print('RUN TAG:', kind, 'STEPS:', len(history))
            print('---------- %s ----------' % 'INITIAL STATE')
            init_state = history[0]
            init_info = parse_info(init_state.info)
            init_info.observation = parse_description(init_state.observation, entities=init_info.entities, verbs=init_info.verbs, ingredients=init_info.recipe.ingredients)
            init_info.score = init_state.score
            init_info.done = init_state.done

            init_info.parsed_admissible_commands_slots = list(zip(init_info.admissible_commands, match_templates(init_info.admissible_commands, init_info.command_templates)))
            init_info.parsed_walkthrough_commands_slots = list(zip(init_info.walkthrough, match_templates(init_info.walkthrough, init_info.command_templates)))
            init_info.parsed_observation_slots = list(zip(init_info.observation.paragraphs, match_templates(restore_placeholder_values(init_info.observation.paragraphs), flatten(init_info.observation.paragraphs))))

            pprint(init_info, width=ppw)
            # print('parsed admissible_commands slots:', match_templates(init_info.admissible_commands, init_info.command_templates))
            # print(match_templates(init_info.admissible_commands, init_info.command_templates))
            # print('---')
            print('restored observation placeholders:', restore_placeholder_values(init_info.observation.paragraphs))
            # print('---')
            # print(match_templates(restore_placeholder_values(init_info.observation.paragraphs), flatten(init_info.observation.paragraphs)))
            # print('---')
            print('---------- %s ----------' % 'START')
            for i in range(1, len(history), 2):
                prv, cmd, nxt = history[i-1:i+2]
                # print(prv)
                print(f'COMMAND: {cmd}')
                # print(nxt)
                # prv/nxt keys: done, info, observation, score
                # info keys: ['description', 'inventory', 'objective', 'entities', 'verbs', 'command_templates', 'admissible_commands', 'extra.recipe', 'extra.walkthrough']
                # print(prv.info.keys())
                # print(prv.info.inventory)

                # print(prv.observation)
                # print(nxt.observation)
                prv_observation = parse_description(prv.observation)
                nxt_observation = parse_description(nxt.observation)
                # print(prv_observation)
                # print(nxt_observation)

                diff_observation = diff(prv_observation, nxt_observation)
                diff_observation = trim_unchanged(diff_observation, True)


                # pprint(Dict(observation=diff_observation))

                prv_info = parse_info(prv.info)
                nxt_info = parse_info(nxt.info)
                # diff_info = Dict({key: diff(prv_info[key], nxt_info[key]) for key in prv_info})
                diff_info = diff(prv_info, nxt_info)

                diff_info.score = trim_unchanged(diff(prv.score, nxt.score))
                diff_info.done = trim_unchanged(diff(prv.done, nxt.done))
                diff_info.observation = diff_observation

                diff_info = trim_unchanged(diff_info, True)

                pprint(diff_info, width=ppw)

                # print(prv_info.description)
                # print(nxt_info)
                # description = prv.info.description
                # description = parse_description(description)
                # inventory = parse_inventory(prv.info.inventory)
                # entities = prv.info.entities
                # verbs = prv.info.verbs
                # command_templates = prv.info.command_templates
                # admissible_commands = prv.info.admissible_commands
                # recipe = prv.info['extra.recipe']
                # recipe = parse_recipe(recipe)
                # walkthrough = prv.info['extra.walkthrough']
                #
                # print(inventory)
                # print(entities)
                # print(verbs)
                # print(command_templates)
                # print(admissible_commands)
                # print(walkthrough)
                # print(recipe)
                # print(description)
            print('---------- %s ----------' % 'END')


    def eval(self) -> None:
        """ Tell the agent it is in evaluation mode. """
        pass  # [You can insert code here.]

    def select_additional_infos(self) -> EnvInfos:
        """
        Returns what additional information should be made available at each game step.

        Requested information will be included within the `infos` dictionary
        passed to `CustomAgent.act()`. To request specific information, create a
        :py:class:`textworld.EnvInfos <textworld.envs.wrappers.filter.EnvInfos>`
        and set the appropriate attributes to `True`. The possible choices are:

        * `description`: text description of the current room, i.e. output of the `look` command;
        * `inventory`: text listing of the player's inventory, i.e. output of the `inventory` command;
        * `max_score`: maximum reachable score of the game;
        * `objective`: objective of the game described in text;
        * `entities`: names of all entities in the game;
        * `verbs`: verbs understood by the the game;
        * `command_templates`: templates for commands understood by the the game;
        * `admissible_commands`: all commands relevant to the current state;

        In addition to the standard information, game specific information
        can be requested by appending corresponding strings to the `extras`
        attribute. For this competition, the possible extras are:

        * `'recipe'`: description of the cookbook;
        * `'walkthrough'`: one possible solution to the game (not guaranteed to be optimal);

        Example:
            Here is an example of how to request information and retrieve it.

            >>> from textworld import EnvInfos
            >>> request_infos = EnvInfos(description=True, inventory=True, extras=["recipe"])
            ...
            >>> env = gym.make(env_id)
            >>> ob, infos = env.reset()
            >>> print(infos["description"])
            >>> print(infos["inventory"])
            >>> print(infos["extra.recipe"])

        Notes:
            The following information *won't* be available at test time:

            * 'walkthrough'

            Requesting additional infos comes with some penalty (called handicap).
            The exact penalty values will be defined in function of the average
            scores achieved by agents using the same handicap.

            Handicap is defined as follows
                max_score, has_won, has_lost,               # Handicap 0
                description, inventory, verbs, objective,   # Handicap 1
                command_templates,                          # Handicap 2
                entities,                                   # Handicap 3
                extras=["recipe"],                          # Handicap 4
                admissible_commands,                        # Handicap 5
        """
        # Get all admissible commands so that this bot has a handicap on the scoreboard.
        # Even though we don't use the commands, we do use prior knowledge so in effect we should have a handicap.
        # return EnvInfos()
        return EnvInfos(admissible_commands=False)

    def _init(self) -> None:
        """ Initialize the agent. """
        self._initialized = True

        # [You can insert code here.]

    def _start_episode(self, obs: List[str], infos: Dict[str, List[Any]]) -> None:
        """
        Prepare the agent for the upcoming episode.

        Arguments:
            obs: Initial feedback for each game.
            infos: Additional information for each game.
        """
        if not self._initialized:
            self._init()

        self._epsiode_has_started = True
        self._game_features: List[Dict] = [defaultdict(lambda: False) for _ in obs]
        self._rooms: List[Dict] = [dict() for _ in obs]
        self._searches: List[Optional[RoomSearch]] = [None for _ in obs]
                   
        self.room_observations = [defaultdict(lambda: False) for _ in obs]
        self.dropped_items_at_room = [defaultdict(set) for _ in obs]

    def _add_features(self, obs: List[str], infos: Dict[str, List[Any]]) -> None:
        """
        Add features for each game.

        Arguments:
            obs: Initial feedback for each game.
            infos: Additional information for each game.
        """
        for game_index,(ob, rdfs) in enumerate(zip(obs, self._game_features)):
                   
            #rooms = self._rooms[game_index]
            #room = None

            # Defaults
            if rdfs[Feature.NUM_ITEMS_HELD] == False:
                rdfs[Feature.NUM_ITEMS_HELD] = 0

            # TODO Optimization: Check if fridge is already open in more ways.
            if "The fridge is empty" in ob:
                rdfs[Feature.OPENED_FRIDGE] = True

            new_room = self._get_room_name(ob)
            changed_room = new_room is not None and new_room != rdfs[Feature.CURRENT_ROOM]
                
            rdfs[Feature.CURRENT_ROOM] = new_room or rdfs[Feature.CURRENT_ROOM]
            #print('CURRENT ROOM:', Feature.CURRENT_ROOM)
            #print('NEW ROOM:', new_room)
            print('CHANGED ROOM:', changed_room)
                   
            if new_room:
                self.current_room = new_room
                if new_room not in self.room_observations[game_index]:
                   self.room_observations[game_index][new_room] = ob
                #elif not self.room_observations[game_index][new_room]:
                #room = rooms.get(new_room)
                #if room and not room.observation:
                #    room.observation = ob
                #    print('ADD OBSERVATION:', ob)
            print('CURRENT ROOM:', self.current_room)

            rdfs[Feature.INVENTORY_SHOWING] = ob.startswith("You are carrying:") \
                                               or ob.startswith("You are carrying nothing.")

            rdfs[Feature.CARRYING_TOO_MUCH] = ob.startswith("You're carrying too many things already.")

            rdfs[Feature.CANT_SEE_SUCH_THING] = ob.startswith("You can't see any such thing.")

            rdfs[Feature.YOU_TAKE] = ob.startswith("You take ")

            rdfs[Feature.NEED_TO_OPEN_FIRST] = _need_to_open_door_pattern.search(ob) is not None

            rdfs[Feature.YOU_OPENED_DOOR] = _you_open_door_pattern.search(ob) is not None

            if rdfs[Feature.CARRYING_TOO_MUCH] or rdfs[Feature.CANT_SEE_SUCH_THING]:
                # Pick up failed.
                rdfs[Feature.NUM_ITEMS_HELD] -= 1

            if ob.startswith("You take the knife "):
                rdfs[Feature.HOLDING_KNIFE] = True
            elif ob.startswith("You drop the knife "):
                rdfs[Feature.HOLDING_KNIFE] = False

            rdfs[Feature.BBQ_PRESENT] = "BBQ" in ob

            rdfs[Feature.OBSERVING_KITCHEN] = "-= Kitchen =-" in ob
            rdfs[Feature.COOKBOOK_PRESENT] = rdfs[Feature.OBSERVING_KITCHEN] and " cookbook" in ob
            rdfs[Feature.COOKBOOK_SHOWING] = "\nIngredients:\n" in ob and "\nDirections:\n" in ob
            if rdfs[Feature.COOKBOOK_SHOWING] and not rdfs[Feature.SEEN_COOKBOOK]:
                rdfs[Feature.SEEN_COOKBOOK] = True
                ingredients, recipe_steps = self._gather_recipe(ob)
                carrying = set(_get_carrying(rdfs))
                for ingredient in ingredients:
                    rdfs[('recipe-ingredient', ingredient)] = True
                    if ingredient not in carrying:
                        rdfs[_ingredient_feat(ingredient)] = True
                    else:
                        rdfs[_ingredient_feat(ingredient)] = False
                        rdfs[('neverdrop', ingredient)] = True
                   
                ingredients_needed = _get_all_required_ingredients(rdfs)
                room_ob = self.room_observations[game_index][self.current_room]
                if room_ob:
                    print('EXTRACTING PRESENT ITEMS FROM SAVED ROOM OBSERVATION, ROOM:', self.current_room)
                    ingredients_present = _get_ingredients_present(room_ob, ingredients_needed)
                    for ingredient in ingredients_present:
                        rdfs[_ingredient_present_feat(ingredient)] = True

                # Remove recipe steps for what we're carrying.
                done_recipe_steps = set()
                for item in carrying:
                    done_recipe_steps.update(_get_recipe_steps_to_make(item))

                for recipe_step_index, recipe_step in enumerate(recipe_steps):
                    if recipe_step not in done_recipe_steps:
                        rdfs[_recipe_step_feat(recipe_step_index, recipe_step)] = True

            if rdfs[Feature.INVENTORY_SHOWING]:
                items = _gather_inventory(ob)
                rdfs[Feature.NUM_ITEMS_HELD] = len(items)
                for item in items:
                    rdfs[_carrying_feat(item)] = True # feat[('carrying', item)] = True
                    rdfs[_ingredient_feat(item)] = False # feat[('ingredient', item)] = False
                    base_ingredient = _base_ingredient(item)
                    if base_ingredient != item:
                        rdfs[_ingredient_feat(base_ingredient)] = False
                        for step in _get_recipe_steps_to_make(item):
                            _remove_recipe_step(rdfs, step)
            elif not rdfs[Feature.COOKBOOK_SHOWING] \
                    and rdfs[Feature.SEEN_COOKBOOK] \
                    and not rdfs[Feature.YOU_TAKE]:
                ingredients_needed = _get_all_required_ingredients(rdfs)
                ingredients_present = _get_ingredients_present(ob, ingredients_needed)
                for ingredient in ingredients_present:
                    rdfs[_ingredient_present_feat(ingredient)] = True

            if rdfs[Feature.SEEN_COOKBOOK] and not rdfs[Feature.FOUND_ALL_INGREDIENTS]:
                # Had before:
                # and not rdfs[Feature.COOKBOOK_SHOWING] \
                # and not rdfs[Feature.INVENTORY_SHOWING]\

                # Discount ingredients we have that can be satisfied by later recipe steps.
                # E.g. "fried carrot" when we have a "carrot" and need to fry later.
                ingredients_to_make = tuple(map(_recipe_step_to_ingredient, _get_recipe_steps(rdfs)))
                ingredients_to_make = set(filter(None, ingredients_to_make))
                ingredients_needed = tuple(
                    set(_get_all_required_ingredients(rdfs))
                    # Had before: - set(_get_all_present_ingredients(rdfs))
                    - ingredients_to_make)
                if len(ingredients_needed) == 0:
                    rdfs[Feature.FOUND_ALL_INGREDIENTS] = True

            if changed_room:
                # Check directions you can go.
                for direction in _directions:
                    present = re.search(r'\b{}\b'.format(direction), ob, re.IGNORECASE) is not None
                    rdfs[_direction_feat(direction)] = present

            # Check closed directions.
            # Clear closed rdfs.
            for direction in _directions:
                if rdfs[_direction_closed_feat(direction)]:
                    del rdfs[_direction_closed_feat(direction)]
            m = _closed_door_pattern.search(ob)
            while m:
                item = m.group('item')
                direction = m.group('direction')
                rdfs[_direction_closed_feat(direction)] = item
                m = _closed_door_pattern.search(ob, pos=m.endpos)

    def _gather_recipe(self, ob):
        ingredients = []
        recipe_steps = []
        in_ingredients = False
        in_recipe_steps = False
        for line in ob.split('\n'):
            if len(line.strip()) == 0:
                in_ingredients = False
                in_recipe_steps = False
                continue
            if line == "Ingredients:":
                in_ingredients = True
                continue
            elif line == "Directions:":
                in_recipe_steps = True
                continue

            if in_ingredients:
                ingredients.append(line.strip())
            elif in_recipe_steps:
                recipe_steps.append(line.strip())

        # Extend.
        for recipe_step in recipe_steps:
            modified_ingredient = _recipe_step_to_ingredient(recipe_step)
            if modified_ingredient is not None:
                ingredients.append(modified_ingredient)

        return ingredients, recipe_steps

    @staticmethod
    def _get_room_name(ob: str) -> Optional[str]:
        result = None
        m = re.search(r'-=\s*([^=]+) =-', ob)
        if m:
            result = m.group(1)
        return result

    def _update_map(self, game_index,
                    prev_room: Room, current_room_name: str, ob: str):
        rooms = self._rooms[game_index]
        room = rooms.get(current_room_name)
        if room is None:
            directions = []
            for direction in _directions:
                if self._game_features[game_index][_direction_feat(direction)]:
                    directions.append(direction)
            room = Room(current_room_name, directions)
            rooms[current_room_name] = room
        if self._searches[game_index] is not None and prev_room is not None:
            prev_dir = self._searches[game_index].prev_direction_traveled
            assert prev_room.name != room.name, f"{prev_room.name}--{prev_dir}-->{room.name}"
            rooms[prev_room.name].directions[prev_dir] = room
            room.directions[opposite_dir(prev_dir)] = prev_room

    def _end_episode(self, obs: List[str], scores: List[int], infos: Dict[str, List[Any]]) -> None:
        """
        Tell the agent the episode has terminated.

        Arguments:
            obs: Previous command's feedback for each game.
            score: The score obtained so far for each game.
            infos: Additional information for each game.
        """
        self._epsiode_has_started = False

        # [You can insert code here.]

    def act(self, obs: List[str], scores: List[int], dones: List[bool], infos: Dict[str, List[Any]]) -> Optional[
        List[str]]:
        """
        Acts upon the current list of observations.

        One text command must be returned for each observation.

        Arguments:
            obs: Previous command's feedback for each game.
            scores: The score obtained so far for each game.
            dones: Whether a game is finished.
            infos: Additional information for each game.

        Returns:
            Text commands to be performed (one per observation).
            If episode had ended (e.g. `all(dones)`), the returned
            value is ignored.

        Notes:
            Commands returned for games marked as `done` have no effect.
            The states for finished games are simply copy over until all
            games are done.
        """
        debug = '--debug' in sys.argv

        if all(dones):
            self._end_episode(obs, scores, infos)
            return  # Nothing to return.

        if not self._epsiode_has_started:
            self._start_episode(obs, infos)

        self._add_features(obs, infos)

        result = []
        for game_index, ob, done, rdfs in zip(range(len(obs)), obs, dones, self._game_features):
            if debug:
                print(ob)
            if done:
                result.append("wait")
                continue
                   
            print('ALL PRESENT:', _get_all_present_ingredients(rdfs))
            print('ALL CARRYING:', _get_carrying(rdfs))
            print('ALL REQ:', _get_all_required_ingredients(rdfs))


            try:
                current_room_name: str = rdfs[Feature.CURRENT_ROOM]
                print('CURRENT ROOM (act):', current_room_name)
                dropped = self.dropped_items_at_room[game_index][current_room_name] if current_room_name else set()
                rooms = self._rooms[game_index]
                   
                if rdfs[Feature.YOU_TAKE]:
                   for item in dropped:
                       if item in rdfs[Feature.YOU_TAKE]:
                           print('PICKED DROPPED ITEM:', item)
                           dropped.remove(item)
                           break


                if self._searches[game_index] is not None:
                    self._searches[game_index].visited.add(current_room_name)
                    prev_room: Room = self._searches[game_index].current_room
                else:
                    prev_room = None
                if not rdfs[Feature.NEED_TO_OPEN_FIRST] and not rdfs[Feature.YOU_OPENED_DOOR]:
                    self._update_map(game_index,
                                     prev_room, current_room_name, ob)
                current_room: Room = rooms[current_room_name]
                   
                if self.take_ingredients == 1:
                    print('TAKE INGREDIENTS BEFORE PREPARE MEAL')
                    norm = lambda item: re.sub(r'(a|an|some|few|the)\s', '', _base_ingredient(item)).strip()
                    carrying = set(_get_carrying(rdfs))
                    print('CARRYING:', carrying)
                    #if 'knife' in carrying:
                    if rdfs[Feature.HOLDING_KNIFE]:
                        result.append('drop knife')
                        continue
                    #ingredients = set(_get_all_present_ingredients(rdfs))
                    ingredients = set(_get_all_recipe_ingredients(rdfs))
                    print('INGREDIENTS:', ingredients)
                    to_take = (ingredients - carrying) & dropped
                    print('TO TAKE:', to_take)
                    if not to_take:
                       self.take_ingredients = 2
                    else:
                       for item in to_take:
                           result.append('take %s' % item)
                           rdfs[_carrying_feat(item)] = True
                           break
                       continue

                if rdfs[Feature.NEED_TO_OPEN_FIRST]:
                    m = _need_to_open_door_pattern.search(ob)
                    assert m
                    print('NEED_TO_OPEN_FIRST')
                    result.append(m.group('task'))
                    continue

                if self._searches[game_index] is not None:
                    if rdfs[Feature.YOU_OPENED_DOOR]:
                        # Optimization: Open the door before getting the error.
                        direction = self._searches[game_index].prev_direction_traveled
                        print('NEED_TO_OPEN_FIRST1')
                        result.append(direction)
                        continue
                    # There is a search in progress.
                    self._searches[game_index].current_room = current_room
                    if self._searches[game_index].target_name == current_room_name:
                        # Found target, stop the search.
                        self._searches[game_index] = None
                    else:
                        # Keep searching.
                        direction = self._searches[game_index].get_next_direction()
                        if direction is not None:
                            print('NEED_TO_OPEN_FIRST2')
                            result.append(direction)
                            continue
                        else:
                            # No path exists.
                            # TODO Need to backtrack to whatever we were doing before.
                            self._searches[game_index] = None

                if rdfs[Feature.CARRYING_TOO_MUCH]:# and (self.reb == 3):
                    # Need to drop something.
                    candidates = tuple(set(_get_carrying(rdfs)) - set(_get_rdfs_with_qualifier(rdfs, 'neverdrop'))
                                       - set(_get_all_required_ingredients(rdfs)))
                    print (_get_carrying(rdfs))
                    print (_get_all_required_ingredients(rdfs))
                    print('neverdrops:', _get_rdfs_with_qualifier(rdfs, "neverdrop"))
                    print ('candidates:', candidates)
                    if not candidates:
                      candidates = _get_rdfs_with_qualifier(rdfs, "neverdrop")
                    assert len(candidates) > 0
                    recipe_steps = _get_recipe_steps(rdfs)
                    assert len(recipe_steps) > 0
                    next_recipe_step = recipe_steps[0]
                    num_tries = 0
                    item = random.choice(candidates)
                    while item in next_recipe_step and num_tries < 10:
                        num_tries += 1
                        item = random.choice(candidates)
                    result.append("drop {}".format(item))
                    dropped.add(item)
                
                   
                    rdfs[_carrying_feat(item)] = False
                    # TODO Note that we might need this later.
                    # if rdfs.get(_ingredient_feat(item)) == False:
                    #     rdfs[_ingredient_feat(item)] = True
                    #     rdfs[Feature.FOUND_ALL_INGREDIENTS] = False
                    rdfs[Feature.NUM_ITEMS_HELD] -= 1
                    continue

                if not rdfs[Feature.SEEN_COOKBOOK] and rdfs[Feature.COOKBOOK_PRESENT]:# and self.reb == 1:
                    result.append("look cookbook")
                    #self.reb = 3
                    continue

                if not rdfs[Feature.DONE_INIT_INVENTORY_CHECK]:
                    result.append("inventory")
                    rdfs[Feature.DONE_INIT_INVENTORY_CHECK] = True
                    continue

                if not rdfs[Feature.SEEN_COOKBOOK]:
                    # Find the Kitchen.
                    self._searches[game_index] = RoomSearch(rooms, current_room, "Kitchen")
                    result.append(self._searches[game_index].get_next_direction())
                    print('SEARCHING FOR KITCHEN')
                    continue

                if current_room_name == "Kitchen" \
                        and not rdfs[Feature.FOUND_ALL_INGREDIENTS] \
                        and not rdfs[Feature.OPENED_FRIDGE]:#  and (self.reb == 2) :
                    rdfs[Feature.OPENED_FRIDGE] = True
                    result.append("open fridge")
                    continue

                # Check if current room has the ingredient.
                if not rdfs[Feature.FOUND_ALL_INGREDIENTS]:# and (self.reb == 2):
                    if rdfs[Feature.NUM_ITEMS_HELD] < _max_capacity:
                        # See if ingredient is here.
                        ingredients_here = tuple(
                            set(_get_all_required_ingredients(rdfs)) & (set(_get_all_present_ingredients(rdfs)) | dropped))
                        if len(ingredients_here) > 0:
                            ingredient = random.choice(ingredients_here)
                            print('SEE IF INGREDIENTS IS HERE')
                            result.append("take {}".format(ingredient))
                            #if ingredient in dropped:
                            #   dropped.remove(ingredient)
                            rdfs[_ingredient_present_feat(ingredient)] = False
                            rdfs[_carrying_feat(ingredient)] = True
                            rdfs[Feature.NUM_ITEMS_HELD] += 1

                            # Remove required ingredients.
                            rdfs[_ingredient_feat(ingredient)] = False
                            base_ingredient = _base_ingredient(ingredient)
                            if base_ingredient != ingredient:
                                rdfs[_ingredient_feat(base_ingredient)] = False
                                for step in _get_recipe_steps_to_make(ingredient):
                                    _remove_recipe_step(rdfs, step)

                            continue

                    if current_room_name == "Kitchen":#  and (self.reb == 2):
                        # Go find ingredients.
                        ingredients_needed = tuple(
                            set(map(_base_ingredient, _get_all_required_ingredients(rdfs)))
                            - set(map(_base_ingredient, _get_carrying(rdfs)))
                            - set(map(_base_ingredient, _get_all_present_ingredients(rdfs))))
                        print('all requried ingredients:', _get_all_required_ingredients(rdfs))
                        print('all present ingredients:', _get_all_present_ingredients(rdfs))
                        print('carrying:', _get_carrying(rdfs))
                        print('ingredients needed:', ingredients_needed)
                        if len(ingredients_needed) == 0:
                           rdfs[Feature.FOUND_ALL_INGREDIENTS] = True
                        assert len(ingredients_needed) > 0
                        direction = None
                        while len(current_room.directions) > 0 and direction is None:
                            ingredient = random.choice(ingredients_needed)
                            room_options = _ingredient_to_rooms[ingredient]
                            room_options = ['Kitchen']
                            for target_room_name in random.sample(room_options, len(room_options)):
                                self._searches[game_index] = RoomSearch(rooms, current_room, target_room_name)
                                direction = self._searches[game_index].get_next_direction()
                                if direction is not None:
                                    break
                                else:
                                    # No path exists.
                                    self._searches[game_index] = None

                        if direction is not None:
                            # Found a direction to go.
                            print('GO FIND ALL INGREDIENTS')
                            result.append(direction)
                            continue

                if not rdfs[Feature.FOUND_ALL_INGREDIENTS] and rdfs[Feature.NUM_ITEMS_HELD] >= _max_capacity:#  and (self.reb == 2):
                    if current_room_name != "Kitchen":
                        # Bring items to Kitchen.
                        self._searches[game_index] = RoomSearch(rooms, current_room, "Kitchen")
                        direction = self._searches[game_index].get_next_direction()
                        print('BRING ITEMS TO KITCHEN')
                        result.append(direction)
                        continue
                    elif not rdfs[Feature.STARTED_COOKING]:
                        # In Kitchen.
                        # Not started cooking.
                        item = random.choice(_get_carrying(rdfs))
                        print('DROPPING IN KITCHEN')
                        result.append("drop {}".format(item))
                        dropped.add(item)
                        rdfs[_carrying_feat(item)] = False
                        # TODO Note that we might need this later.
                        # if rdfs.get(_ingredient_feat(item)) == False:
                        #     rdfs[_ingredient_feat(item)] = True
                        #     rdfs[Feature.FOUND_ALL_INGREDIENTS] = False

                        rdfs[Feature.NUM_ITEMS_HELD] -= 1
                        continue

                # Go through recipe steps.
                recipe_steps = _get_recipe_steps(rdfs)
                #self.reb = 3
                if len(recipe_steps) > 0:#  and (self.reb == 3):
                    next_recipe_step = recipe_steps[0]
                    if _requires_knife(next_recipe_step) and not rdfs[Feature.HOLDING_KNIFE]:
                        if rdfs[Feature.NUM_ITEMS_HELD] < _max_capacity:
                            result.append("take knife")
                            rdfs[Feature.NUM_ITEMS_HELD] += 1
                            continue
                        else:
                            # Need to drop something.
                            candidates = tuple(set(_get_carrying(rdfs)) - set(_get_all_required_ingredients(rdfs)))
                            assert len(candidates) > 0
                            item = random.choice(candidates)
                            num_tries = 0
                            while item in next_recipe_step and num_tries < 10:
                                num_tries += 1
                                item = random.choice(candidates)
                            print('DROP NEVIETA')
                            result.append("drop {}".format(item))
                            dropped.add(item)
                            rdfs[_carrying_feat(item)] = False
                            # TODO Note that we might need this later.
                            # if rdfs.get(_ingredient_feat(item)) == False:
                            #     rdfs[_ingredient_feat(item)] = True
                            #     rdfs[Feature.FOUND_ALL_INGREDIENTS] = False
                            rdfs[Feature.NUM_ITEMS_HELD] -= 1
                            continue

                    if _grill_pattern.match(next_recipe_step) and not rdfs[Feature.BBQ_PRESENT]:
                        # Go to the BBQ in the Backyard.
                        self._searches[game_index] = RoomSearch(rooms, current_room, "Backyard")
                        direction = self._searches[game_index].get_next_direction()
                        print('GO TO BBQ 0')
                        result.append(direction)
                        continue
                    elif (_fry_pattern.match(next_recipe_step)
                          or _roast_pattern.match(next_recipe_step)
                          or next_recipe_step == "prepare meal") \
                            and current_room_name != "Kitchen":
                        self._searches[game_index] = RoomSearch(rooms, current_room, "Kitchen")
                        direction = self._searches[game_index].get_next_direction()
                        print('GO TO BBQ 1')
                        result.append(direction)
                        continue
                   
                   
                    norm = lambda item: re.sub(r'(a|an|some|few|the)\s', '', _base_ingredient(item)).strip()
                   
                    if _requires_knife(next_recipe_step) and rdfs[Feature.HOLDING_KNIFE]:
                       items_available = dropped | set(_get_carrying(rdfs)) | set(_get_all_present_ingredients(rdfs))
                       # try to skip slice & dice steps
                       base_ingredient = re.sub(r'(a|an|some|few|the)\s', '', _base_ingredient(next_recipe_step)).strip()
                       print('BASE INGREDIENT:', base_ingredient)
                       #next_recipe_step = recipe_steps[0]
                       print('DROPPED ITEMS:', ', '.join(dropped))
                       print('ALL ITEMS AVAILABLE:', ', '.join(items_available))
                       print('RECIPE STEPS:', recipe_steps)
                       remove_step = False
                       for step in recipe_steps[1:]:
                           print('STEP:', step)
                           if base_ingredient == _base_ingredient(step):
                               print('POSSIBLE MATCH:', step)
                               print('REMOVE CURRENT STEP:', next_recipe_step)
                               _remove_recipe_step(rdfs, next_recipe_step)
                               remove_step = True
                               break
                           continue
                           for item in items_available:
                               print('CHECK AGAINST:', _base_ingredient(item))
                               if base_ingredient == _base_ingredient(item):
                                   print('POSSIBLE MATCH:', item)
                                   print('REMOVE CURRENT STEP:', next_recipe_step)
                                   _remove_recipe_step(rdfs, next_recipe_step)
                                   remove_step = True
                                   break
                           if remove_step:
                               break
                       if remove_step:
                           continue
                                   
                       for item in dropped:
                           print('CHECKING DROPPED ITEM:', item, 'AGAINST:', _commandify_recipe_step(next_recipe_step), 'OR:', next_recipe_step)
                           print(norm(item), '!=', norm(_commandify_recipe_step(next_recipe_step)))
                           if norm(item) in norm(_commandify_recipe_step(next_recipe_step)):
                               result.append('take %s' % item)
                               #dropped.remove(item)
                               print('TRY TO PICK DROPPED REQUIRED ITEM', item)
                               continue
                       
                    if next_recipe_step == "prepare meal" and self.take_ingredients == 0:
                       self.take_ingredients = 1
                       print('TAKE INGREDIENTS 1st')
                       result.append('wait')
                       continue

                    # TODO Maybe check if the ingredient is already modified.
                    result.append(_commandify_recipe_step(next_recipe_step))
                    _remove_recipe_step(rdfs, next_recipe_step)
                    rdfs[Feature.STARTED_COOKING] = True
                    # TODO Maybe remove from required ingredients (remove ingredient feature).
                    continue
                elif self.take_ingredients == 0:
                    self.take_ingredients = 1
                    continue
                if self.take_ingredients == 2:
                    result.append("eat meal")
                    continue
                if (self.reb == 3):
                    # Done
                    print(self.reb)
                    result.append("eat meal")
                    print(result)
                    self.reb = 4
                    continue
                elif (self.reb == 4):
                    # Done
                    result.append("quit")
                    self.reb = 5
                    continue
            except:
                if debug:
                    logging.exception("Will wait.")
            result.append(None)

        result = ["wait" if r is None else r for r in result]
        if debug:
            print(f"ACT: \"{result[-1]}\"")

        return result

In [5]:

#gamefile = gamefiles[0]  # Pick a game.

selected_gamefiles = list(filter(lambda fn: 'recipe2+cut+open+drop+go6' in fn, gamefiles))
#selected_gamefiles = list(filter(lambda fn: 'recipe2+cook+go9' in fn, gamefiles))
#selected_gamefiles = list(filter(lambda fn: 'recipe2+cut+drop+go6' in fn, gamefiles))
#selected_gamefiles = list(filter(lambda fn: 'recipe3+cut' in fn, gamefiles))
#selected_gamefiles = list(filter(lambda fn: 'recipe3+take3' in fn, gamefiles))
#selected_gamefiles = list(filter(lambda fn: 'recipe3+take3+cook+open+drop+go12' in fn, gamefiles))
#selected_gamefiles = list(filter(lambda fn: 'recipe3+cook+cut+open+go12' in fn, gamefiles))
#ok: selected_gamefiles = list(filter(lambda fn: 'recipe3+cut+go9' in fn, gamefiles))
#print(selected_gamefiles)

#for gamen, gamefile in enumerate(gamefiles[0:7]):
for gamen, gamefile in enumerate(selected_gamefiles):
#    if gamen < 6:
#        continue

    print('============== GAME %i ( %s )' % (gamen, gamefile))

    requested_infos = EnvInfos(description=True, inventory=True, extras=["recipe", "walkthrough"])
    env_id = textworld.gym.register_games([gamefile], requested_infos)
    env_id = textworld.gym.make_batch(env_id, batch_size=1)

    #agent = textworld.agents.HumanAgent()

    #agent = agent_class()


    env = gym.make(env_id)



    obs, infos = env.reset()
    
    print(infos['extra.walkthrough'])


    env.render()
    all_commands = []
    scores = [0] * len(obs)
    dones = [False] * len(obs)
    steps = [0] * len(obs)

    #_start_episode(self, obs, infos)
    agent = CustomAgent()


    while not all(dones):
                # Increase step counts.
                steps = [step + int(not done) for step, done in zip(steps, dones)]

                commands = agent.act(obs, scores, dones, infos)
                print (commands)
                all_commands.append(commands)
                obs, scores, dones, infos = env.step(commands)
                env.render()

    # Let the agent knows the game is done.
    agent.act(obs, scores, dones, infos)
    
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')
    print('=============================================================================================')


============== GAME 0 ( /root/notebooks/Guntis/train/tw-cooking-recipe2+cut+open+drop+go6-Yxas3qxS6PJU057.ulx )
[['go north', 'go east', 'go east', 'drop salt', 'take knife from counter', 'slice pork chop with knife', 'drop knife', 'take salt', 'prepare meal', 'eat meal']]



                    ________  ________  __    __  ________
                   |        \|        \|  \  |  \|        \
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$
                      | $$   | $$__     \$$\/  $$   | $$
                      | $$   | $$  \     >$$  $$    | $$
                      | $$   | $$$$$    /  $$$$\    | $$
                      | $$   | $$_____ |  $$ \$$\   | $$
                      | $$   | $$     \| $$  | $$   | $$
                       \$$    \$$$$$$$$ \$$   \$$    \$$
              __       __   ______   _______   __        _______
             |  \  _  |  \ /      \ |       \ |  \      |       \
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
    

[['drop red potato', 'drop salt', 'take knife from counter', 'slice purple potato with knife', 'drop knife', 'take salt', 'prepare meal', 'eat meal']]



                    ________  ________  __    __  ________
                   |        \|        \|  \  |  \|        \
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$
                      | $$   | $$__     \$$\/  $$   | $$
                      | $$   | $$  \     >$$  $$    | $$
                      | $$   | $$$$$    /  $$$$\    | $$
                      | $$   | $$_____ |  $$ \$$\   | $$
                      | $$   | $$     \| $$  | $$   | $$
                       \$$    \$$$$$$$$ \$$   \$$    \$$
              __       __   ______   _______   __        _______
             |  \  _  |  \ /      \ |       \ |  \      |       \
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$

[['drop red onion', 'go north', 'go north', 'drop white onion', 'take knife from counter', 'slice red apple with knife', 'drop knife', 'take white onion', 'drop red apple', 'take knife', 'chop white onion with knife', 'drop knife', 'take red apple', 'prepare meal', 'eat meal']]



                    ________  ________  __    __  ________
                   |        \|        \|  \  |  \|        \
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$
                      | $$   | $$__     \$$\/  $$   | $$
                      | $$   | $$  \     >$$  $$    | $$
                      | $$   | $$$$$    /  $$$$\    | $$
                      | $$   | $$_____ |  $$ \$$\   | $$
                      | $$   | $$     \| $$  | $$   | $$
                       \$$    \$$$$$$$$ \$$   \$$    \$$
              __       __   ______   _______   __        _______
             |  \  _  |  \ /      \ |       \ |  \      |       \
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\

> drop knife
You drop the knife on the ground.

CHANGED ROOM: False
CURRENT ROOM: Kitchen
ALL PRESENT: ['red apple']
ALL CARRYING: ['white onion']
ALL REQ: ['red apple', 'sliced red apple', 'chopped white onion']
CURRENT ROOM (act): Kitchen
TAKE INGREDIENTS BEFORE PREPARE MEAL
CARRYING: {'white onion'}
INGREDIENTS: {'white onion', 'red apple', 'sliced red apple', 'chopped white onion', 'roasted red apple'}
TO TAKE: {'roasted red apple'}
['take roasted red apple']
> take roasted red apple
You pick up the red apple from the ground.

CHANGED ROOM: False
CURRENT ROOM: Kitchen
ALL PRESENT: ['red apple']
ALL CARRYING: ['white onion', 'roasted red apple']
ALL REQ: ['red apple', 'sliced red apple', 'chopped white onion']
CURRENT ROOM (act): Kitchen
TAKE INGREDIENTS BEFORE PREPARE MEAL
CARRYING: {'white onion', 'roasted red apple'}
INGREDIENTS: {'white onion', 'red apple', 'sliced red apple', 'chopped white onion', 'roasted red apple'}
TO TAKE: set()
['prepare meal']
> prepare meal
The recipe r

[['drop red hot pepper', 'drop red potato', 'go east', 'go north', 'drop yellow potato', 'take knife from counter', 'chop yellow bell pepper with knife', 'drop knife', 'take yellow potato', 'drop yellow bell pepper', 'take knife', 'chop yellow potato with knife', 'drop knife', 'take yellow bell pepper', 'prepare meal', 'eat meal']]



                    ________  ________  __    __  ________
                   |        \|        \|  \  |  \|        \
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$
                      | $$   | $$__     \$$\/  $$   | $$
                      | $$   | $$  \     >$$  $$    | $$
                      | $$   | $$$$$    /  $$$$\    | $$
                      | $$   | $$_____ |  $$ \$$\   | $$
                      | $$   | $$     \| $$  | $$   | $$
                       \$$    \$$$$$$$$ \$$   \$$    \$$
              __       __   ______   _______   __        _______
             |  \  _  |  \ /      \ |       \ |  \      |       \
           

...

[['drop red apple', 'drop orange bell pepper', 'go north', 'go north', 'drop red hot pepper', 'take knife from counter', 'slice banana with knife', 'drop knife', 'take red hot pepper', 'drop banana', 'take knife', 'slice red hot pepper with knife', 'drop knife', 'take banana', 'prepare meal', 'eat meal']]



                    ________  ________  __    __  ________
                   |        \|        \|  \  |  \|        \
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$
                      | $$   | $$__     \$$\/  $$   | $$
                      | $$   | $$  \     >$$  $$    | $$
                      | $$   | $$$$$    /  $$$$\    | $$
                      | $$   | $$_____ |  $$ \$$\   | $$
                      | $$   | $$     \| $$  | $$   | $$
                       \$$    \$$$$$$$$ \$$   \$$    \$$
              __       __   ______   _______   __        _______
             |  \  _  |  \ /      \ |       \ |  \      |       \
             | $$ / \ | $$|  $$$$$$\| 

> take red hot pepper
You're carrying too many things already.

CHANGED ROOM: False
CURRENT ROOM: Kitchen
ALL PRESENT: []
ALL CARRYING: ['banana']
ALL REQ: ['sliced banana', 'sliced red hot pepper']
CURRENT ROOM (act): Kitchen
['banana']
['sliced banana', 'sliced red hot pepper']
neverdrops: ['banana', 'red hot pepper']
candidates: ()
['drop red hot pepper']
> drop red hot pepper
The red hot pepper is already here.

CHANGED ROOM: False
CURRENT ROOM: Kitchen
ALL PRESENT: []
ALL CARRYING: ['banana']
ALL REQ: ['sliced banana', 'sliced red hot pepper']
CURRENT ROOM (act): Kitchen
TAKE INGREDIENTS 1st
['wait']
> wait
Time passes.

CHANGED ROOM: False
CURRENT ROOM: Kitchen
ALL PRESENT: []
ALL CARRYING: ['banana']
ALL REQ: ['sliced banana', 'sliced red hot pepper']
CURRENT ROOM (act): Kitchen
TAKE INGREDIENTS BEFORE PREPARE MEAL
CARRYING: {'banana'}
['drop knife']
> drop knife
You drop the knife on the ground.

CHANGED ROOM: False
CURRENT ROOM: Kitchen
ALL PRESENT: []
ALL CARRYING: ['banana']

[['drop cilantro', 'take knife from table', 'dice parsley with knife', 'drop knife', 'take cilantro', 'prepare meal', 'eat meal']]



                    ________  ________  __    __  ________
                   |        \|        \|  \  |  \|        \
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$
                      | $$   | $$__     \$$\/  $$   | $$
                      | $$   | $$  \     >$$  $$    | $$
                      | $$   | $$$$$    /  $$$$\    | $$
                      | $$   | $$_____ |  $$ \$$\   | $$
                      | $$   | $$     \| $$  | $$   | $$
                       \$$    \$$$$$$$$ \$$   \$$    \$$
              __       __   ______   _______   __        _______
             |  \  _  |  \ /      \ |       \ |  \      |       \
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$  | $$
             

> slice the red apple
(with the knife)
You slice the red apple.



Your score has just gone up by one point.

CHANGED ROOM: False
CURRENT ROOM: Kitchen
ALL PRESENT: ['red hot pepper']
ALL CARRYING: ['red apple']
ALL REQ: ['red hot pepper', 'sliced red apple']
CURRENT ROOM (act): Kitchen
TAKE INGREDIENTS 1st
['wait']
> wait
Time passes.

CHANGED ROOM: False
CURRENT ROOM: Kitchen
ALL PRESENT: ['red hot pepper']
ALL CARRYING: ['red apple']
ALL REQ: ['red hot pepper', 'sliced red apple']
CURRENT ROOM (act): Kitchen
TAKE INGREDIENTS BEFORE PREPARE MEAL
CARRYING: {'red apple'}
['drop knife']
> drop knife
You drop the knife on the ground.

CHANGED ROOM: False
CURRENT ROOM: Kitchen
ALL PRESENT: ['red hot pepper']
ALL CARRYING: ['red apple']
ALL REQ: ['red hot pepper', 'sliced red apple']
CURRENT ROOM (act): Kitchen
TAKE INGREDIENTS BEFORE PREPARE MEAL
CARRYING: {'red apple'}
INGREDIENTS: {'roasted red hot pepper', 'sliced red apple', 'red hot pepper', 'red apple'}
TO TAKE: {'roasted red hot pe

> take fried red apple
You're carrying too many things already.

CHANGED ROOM: False
CURRENT ROOM: Kitchen
ALL PRESENT: []
ALL CARRYING: ['red hot pepper']
ALL REQ: ['sliced red apple', 'chopped yellow bell pepper']
CURRENT ROOM (act): Kitchen
['red hot pepper']
['sliced red apple', 'chopped yellow bell pepper']
neverdrops: []
candidates: ('red hot pepper',)
['drop red hot pepper']
> drop red hot pepper
You drop the red hot pepper on the ground.

CHANGED ROOM: False
CURRENT ROOM: Kitchen
ALL PRESENT: []
ALL CARRYING: []
ALL REQ: ['sliced red apple', 'chopped yellow bell pepper']
CURRENT ROOM (act): Kitchen
BASE INGREDIENT: yellow bell pepper
DROPPED ITEMS: yellow potato, fried red apple, red potato, red hot pepper, fried yellow bell pepper
ALL ITEMS AVAILABLE: yellow potato, fried red apple, red hot pepper, fried yellow bell pepper, red potato
RECIPE STEPS: ['chop the yellow bell pepper', 'prepare meal']
STEP: prepare meal
CHECKING DROPPED ITEM: yellow potato AGAINST: chop the yellow b

## Playing a game 

## Visualizing a `TextWorld.Game` object